In [1]:
import tensorflow as tf
import numpy as np

/Users/olaf/.pyenv/versions/anaconda3/envs/tfp3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = np.loadtxt('data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [3]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [20]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

tf.summary.scalar('cost', cost)

<tf.Tensor 'cost:0' shape=() dtype=string>

In [23]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    save.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    print('Step %d' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X:x_data, Y:y_data}))

summary = sess.run(merged, feed_dict={X:x_data, Y:y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

Step 1 Cost: 1.660
Step 2 Cost: 1.497
Step 3 Cost: 1.353
Step 4 Cost: 1.228
Step 5 Cost: 1.123
Step 6 Cost: 1.036
Step 7 Cost: 0.966
Step 8 Cost: 0.908
Step 9 Cost: 0.861
Step 10 Cost: 0.823
Step 11 Cost: 0.792
Step 12 Cost: 0.765
Step 13 Cost: 0.742
Step 14 Cost: 0.722
Step 15 Cost: 0.705
Step 16 Cost: 0.690
Step 17 Cost: 0.676
Step 18 Cost: 0.664
Step 19 Cost: 0.653
Step 20 Cost: 0.643
Step 21 Cost: 0.634
Step 22 Cost: 0.626
Step 23 Cost: 0.618
Step 24 Cost: 0.611
Step 25 Cost: 0.605
Step 26 Cost: 0.599
Step 27 Cost: 0.594
Step 28 Cost: 0.590
Step 29 Cost: 0.586
Step 30 Cost: 0.582
Step 31 Cost: 0.578
Step 32 Cost: 0.576
Step 33 Cost: 0.573
Step 34 Cost: 0.570
Step 35 Cost: 0.568
Step 36 Cost: 0.566
Step 37 Cost: 0.565
Step 38 Cost: 0.563
Step 39 Cost: 0.562
Step 40 Cost: 0.561
Step 41 Cost: 0.560
Step 42 Cost: 0.559
Step 43 Cost: 0.558
Step 44 Cost: 0.558
Step 45 Cost: 0.557
Step 46 Cost: 0.557
Step 47 Cost: 0.556
Step 48 Cost: 0.556
Step 49 Cost: 0.555
Step 50 Cost: 0.555
Step 51 C

In [24]:
saver.save(sess, 'model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data, Y:y_data}))
print('실제값:', sess.run(target, feed_dict={X:x_data, Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X:x_data, Y:y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
